<a href="https://colab.research.google.com/github/john-caballero/Perfilamiento-basico/blob/main/sesion2/Laboratorio_2_%E2%80%94_Correlacion_Asociacion_y_Arboles_de_decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Laboratorio 2 — Correlación, Asociación y Árboles de Decisión**

---

**Objetivos**

- Medir y visualizar relaciones entre variables numéricas y categóricas.

- Identificar variables asociadas al riesgo o valor del cliente.

- Construir un árbol de decisión interpretativo para explicar un outcome (ej. default o compra).

- Traducir hallazgos en reglas de negocio o segmentos accionables.

**Índice**

- Cargar datos y preprocesamiento

- Correlaciones entre variables numéricas

- Asociación entre variables categóricas

- Árbol de decisión interpretativo

- Importancia y reglas extraídas

- Insights y decisiones

## 1) Cargar datos y preprocesamiento

In [ ]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# df = pd.read_csv("tu_archivo.csv")  # o read_excel(...)
df = df.copy()  # si ya está cargado

target = "DEF12"  # o la variable objetivo relevante
num_cols = df.select_dtypes("number").columns.tolist()
cat_cols = [c for c in df.select_dtypes("object").columns if df[c].nunique() < 20]

df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
print(df[num_cols].describe(percentiles=[.25,.5,.75]).T)


## 2) Correlaciones entre variables numéricas



**Concepto:** mide la fuerza y dirección lineal o monótona de la relación entre dos variables.

In [ ]:
corr_pearson = df[num_cols].corr(method="pearson")
corr_spearman = df[num_cols].corr(method="spearman")

plt.figure(figsize=(10,6))
sns.heatmap(corr_spearman, cmap="coolwarm", center=0)
plt.title("Correlación (Spearman)")
plt.show()

# Top correlaciones con la variable objetivo
target_corr = corr_spearman[target].sort_values(ascending=False)
target_corr.head(10)


**Interpretación:**

Correlaciones positivas (+) → variables que crecen juntas.

Correlaciones negativas → relaciones inversas.

Ideal discutir casos donde hay alta correlación entre predictores (riesgo de multicolinealidad).

## 3) Asociación entre variables categóricas

**Concepto:** mide dependencia entre variables cualitativas (no ordinales).

In [ ]:
from scipy.stats import chi2_contingency
import itertools

def cramers_v(x, y):
    tabla = pd.crosstab(x, y)
    chi2 = chi2_contingency(tabla)[0]
    n = tabla.sum().sum()
    phi2 = chi2 / n
    r, k = tabla.shape
    return np.sqrt(phi2 / (min(k-1, r-1)))

asoc = []
for c1, c2 in itertools.combinations(cat_cols, 2):
    asoc.append((c1, c2, cramers_v(df[c1], df[c2])))
asoc_df = pd.DataFrame(asoc, columns=["Var1","Var2","CramérV"]).sort_values("CramérV", ascending=False)
asoc_df.head(10)


**Interpretación:**

Cramér’s V mide la intensidad de asociación (0 = ninguna, 1 = perfecta).

Ideal para variables de tipo “segmento”, “producto”, “canal”.

## 4) Árbol de decisión interpretativo


**Concepto:** un árbol divide el espacio de variables en reglas que maximizan la pureza de los grupos (criterio Gini o Entropía).

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split

X = df[num_cols].drop(columns=[target], errors="ignore").fillna(0)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree = DecisionTreeClassifier(max_depth=4, min_samples_leaf=100, random_state=42)
tree.fit(X_train, y_train)

plt.figure(figsize=(18,8))
plot_tree(tree, feature_names=X.columns, class_names=["NoDefault","Default"], filled=True, fontsize=10)
plt.title("Árbol de Decisión - Drivers del Default")
plt.show()


**Importancia y reglas extraídas**

In [ ]:
imp = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": tree.feature_importances_
}).sort_values("Importancia", ascending=False)
imp.head(10)

from sklearn.metrics import classification_report
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))


**Lectura:**

- Las variables con mayor importancia explican mejor la variación del target.

- Las ramas del árbol equivalen a reglas automáticas de segmentación, útiles para campañas o políticas de riesgo.

**Tarea en clase**

- Selecciona 2 variables con alta correlación y explica su relación.

- Identifica una asociación categórica relevante.

- Interpreta las 3 primeras reglas del árbol como segmentos con posible acción (campaña, precio, validación).

---

# Gracias por completar este laboratorio!

---
